# Pilot Scheduling
## Using PuLP with pandas and binary constraints to solve a scheduling problem

In this example, we'll be solving a scheduling problem. We have a number of pilots and 9 different go's to fill

We want to produce a schedule of pilots from both plants that meets our demand with the lowest cost.

A pilot can be in 2 states:
* Off - not flying
* On - flying

Pilots are either available or not available for each go.

Need an optimization function

In [1]:
import pandas as pd
import pulp
import math
import time

In [2]:
pilots = pd.read_csv('csv/pilot_availability_real.csv',index_col=['PILOT'])
# pilots

In [3]:
pilot_quals = pd.read_csv('csv/pilot_qual_real.csv',index_col=['PILOT'])
# pilot_quals

We'll also import our demand data

In [4]:
lines = pd.read_csv('csv/go_demand_real.csv', index_col=['GO','Line'])
# lines

Pilot status is modelled as a binary variable. It will have a value of 1 if the pilot is flying and a value of 0 when the pilot is off.

Binary variables are the same as integer variables but constrained to be >= 0 and <=1

Again this has a value for each month for each factory, again given by the index of our DataFrame

In [5]:
# for go,line in lines.index:
#     for pilot in pilots.index:
#         if(pilots.loc[pilot][go-1]):
#             print("%s is unavailable for %d"%(pilot,line))

In [5]:
pilot_status = pulp.LpVariable.dicts("pilot_status",
                                     ((Line,PILOT) for Line  in lines.index for PILOT in pilots.index ),
                                     cat='Binary')

In [6]:
type(pilot_status)

dict

In [7]:
ground_duty = pulp.LpVariable.dicts("ground_duty",
                                     ((GO,PILOT) for GO  in range(1,10) for PILOT in pilots.index ),
                                     cat='Binary')

In [8]:
type(ground_duty)

dict

We instantiate our model and use LpMinimize as the aim is to minimise costs.

In [9]:
model = pulp.LpProblem("PilotMinSchedProb", pulp.LpMaximize)

In our objective function we include our 2 costs: 
* Our variable costs is the product of the variable costs per unit and production
* Our fixed costs is the factory status - 1 (on) or 0 (off) - multiplied by the fixed cost of production

In [10]:
model += pulp.lpSum(sum(sum(lines.loc[(go1,line1),'Requirement']*pilot_status[((go1,line1),pilot)]-
        lines.loc[(go2,line2),'Requirement']*pilot_status[((go2,line2),pilot)]
        for go1,line1 in lines.index for go2,line2 in lines.index if go2 < go1) for pilot in pilots.index))

In [11]:
# total=0
# count=0
# for i in range(10):
#     time1=time.process_time()
#     model += pulp.lpSum(sum(sum(lines.loc[(go1,line1),'Requirement']*pilot_status[((go1,line1),pilot)]-
#             lines.loc[(go2,line2),'Requirement']*pilot_status[((go2,line2),pilot)]
#             for go1,line1 in lines.index for go2,line2 in lines.index if go2 < go1) for pilot in pilots.index))
#     total+=time.process_time()-time1
#     count+=1
# print(total/count)

We build up our constraints

In [12]:
for i in range(1,10):
    model+= sum(ground_duty[i,x]*pilot_quals.loc[x,'TOP3'] for x in pilots.index) == 1
    model+= sum(ground_duty[i,x] for x in pilots.index) == 1

In [13]:
for line in lines.index:
    model += sum(pilot_status[(line,x)]*
                 pilot_quals.loc[x,'QUAL'] for x in pilots.index) >= lines.loc[line, 'Requirement']

In [14]:
for line in lines.index:
    model += sum(pilot_status[(line,x)] for x in pilots.index) == 1 #only one pilot flying each line

In [15]:
for pilot in pilots.index:
    for go in range(1,10):
        model += (sum((pilot_status[((go2,line2),pilot)] for go2,line2 in lines.index if go2==go)) + ground_duty[go,pilot]) <= 1
        #pilot can only fly or do a ground duty once each go

In [16]:
for pilot in pilots.index:
    for go1,line1 in lines.index:
        model += pilot_status[((go1,line1),pilot)]+sum(pilot_status[((go2,line2),pilot)] 
                                                       for go2,line2 in lines.index if go2==(go1+1)) <= 1 
        #pilots cannot fly consecutive go's

In [17]:
for go,line in lines.index:
    for pilot in pilots.index:
        if(pilots.loc[pilot][go-1]):
            model += pilot_status[((go,line),pilot)] == 0 #pilot is unavailable for that line

In [18]:
# model

We then solve the model

In [19]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [20]:
# time=0
# count=0
# for i in range(500):
#     model.solve()
#     time+= model.solutionTime
#     count+=1

# print(time/count)

Let's take a look at the optimal production schedule output for each month from each factory. For ease of viewing we'll output the data to a pandas DataFrame.

In [21]:
output = []
for Line, PILOT in pilot_status:
    if (pilot_status[(Line,PILOT)].varValue):
        var_output = {
            'Go':Line[0],
            'Line': Line[1],
            'PILOT': PILOT,
        }
        output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Line'])
output_df.set_index(['Go','Line'], inplace=True)

In [22]:
ground_sched = []
for Line, PILOT in ground_duty:
    if (ground_duty[(Line,PILOT)].varValue):
        var_output = {
            'Go':Line,
            'PILOT': PILOT
        }
        ground_sched.append(var_output)
ground_sched_df = pd.DataFrame.from_records(ground_sched).sort_values(['Go'])
ground_sched_df.set_index(['Go'], inplace=True)

In [23]:
pd.options.display.max_rows = None

In [24]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [25]:
ground_sched_df

,PILOT
Go,
1,PITBULL
2,JACKAL
3,HOOPS
4,CLIP
5,COPEN
6,DUKE
7,SKIDS
8,WORM
9,HOOPS


In [26]:
display_side_by_side(output_df.loc[1:2,:],
                    output_df.loc[3:4,:],
                    output_df.loc[5:6,:],
                    output_df.loc[7:8,:],
                    output_df.loc[9,:])

PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 1 
 101 
 JACKAL 
 
 
 102 
 COPEN 
 
 
 103 
 DUKE 
 
 
 104 
 IRISH 
 
 
 105 
 REBEL 
 
 
 106 
 HEIST 
 
 
 107 
 SHADOW 
 
 
 108 
 WORM 
 
 
 2 
 201 
 BOND 
 
 
 202 
 SHINER 
 
 
 203 
 LITTLE 
 
 
 204 
 PITBULL 
 
 
 205 
 BUSTER 
 
 
 206 
 COACH 
 
 
 
 
 
 
 
 PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 3 
 301 
 HOLSTER 
 
 
 302 
 CLIP 
 
 
 303 
 HATTRICK 
 
 
 304 
 REBEL 
 
 
 305 
 TAOS 
 
 
 306 
 NAPE 
 
 
 307 
 SHADOW 
 
 
 308 
 COPEN 
 
 
 4 
 401 
 LITTLE 
 
 
 402 
 HEAT 
 
 
 403 
 DUKE 
 
 
 404 
 BOND 
 
 
 405 
 PITBULL 
 
 
 406 
 BUSTER 
 
 
 
 
 
 
 
 PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 5 
 501 
 FLIP 
 
 
 502 
 KONG 
 
 
 503 
 WORM 
 
 
 504 
 CLIP 
 
 
 505 
 HOOPS 
 
 
 506 
 COACH 
 
 
 507 
 NAPE 
 
 
 508 
 IRISH 
 
 
 6 
 601 
 COPEN 
 
 
 602 
 TAOS 
 
 
 603 
 HATTRICK 
 
 
 604 
 FACE 
 
 
 605 
 JACKAL 
 
 
 606 
 SKIDS 
 
 
 
 
 
 
 
 PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 7 
 701 
 DUKE 
 
 
 702 
 IRISH 
 
 
 703 
 PITBULL 
 
 
 704 
 CLIP 
 
 
 705 
 LITTLE 
 
 
 706 
 SHADOW 
 
 
 707 
 NAPE 
 
 
 708 
 AMP 
 
 
 8 
 801 
 TAOS 
 
 
 802 
 SHINER 
 
 
 803 
 COPEN 
 
 
 804 
 SKIDS 
 
 
 805 
 BUSTER 
 
 
 806 
 CASS 
 
 
 
 
 
 
 PILOT 
 
 
 Line 
 
 
 
 
 
 901 
 CLIP 
 
 
 902 
 REBEL 
 
 
 903 
 WORM 
 
 
 904 
 PITBULL 
 
 
 905 
 JACKAL 
 
 
 906 
 BOND 
 
 
 907 
 HOLSTER 
 
 
 908 
 NAPE

In [27]:
model.solve

<bound method LpProblem.solve of PilotMinSchedProb:
MAXIMIZE
-112*pilot_status_((1,_101),_'AMP') + -112*pilot_status_((1,_101),_'APE') + -112*pilot_status_((1,_101),_'BOND') + -112*pilot_status_((1,_101),_'BUSTER') + -112*pilot_status_((1,_101),_'CASS') + -112*pilot_status_((1,_101),_'CLIP') + -112*pilot_status_((1,_101),_'COACH') + -112*pilot_status_((1,_101),_'COPEN') + -112*pilot_status_((1,_101),_'CRUTCH') + -112*pilot_status_((1,_101),_'DUKE') + -112*pilot_status_((1,_101),_'FACE') + -112*pilot_status_((1,_101),_'FLIP') + -112*pilot_status_((1,_101),_'HATTRICK') + -112*pilot_status_((1,_101),_'HEAT') + -112*pilot_status_((1,_101),_'HEIST') + -112*pilot_status_((1,_101),_'HOLSTER') + -112*pilot_status_((1,_101),_'HOOPS') + -112*pilot_status_((1,_101),_'IRISH') + -112*pilot_status_((1,_101),_'JACKAL') + -112*pilot_status_((1,_101),_'KONG') + -112*pilot_status_((1,_101),_'LITTLE') + -112*pilot_status_((1,_101),_'NAPE') + -112*pilot_status_((1,_101),_'PHANTOM') + -112*pilot_status_((1

In [28]:
output = []
for Line, PILOT in pilot_status:
    if (pilot_status[(Line,PILOT)].varValue):
        var_output = {
            'Go':Line[0],
            'Line': Line[1],
            'PILOT': PILOT,
        }
        output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Line'])
output_df.set_index(['Go','Line'], inplace=True)

In [29]:
sum(sum((1/(abs(go2-go1)+1))*
             pilot_status[(go1,line1),pilot].varValue*
             lines.loc[(go1,line1),'Requirement']*
             pilot_status[((go2,line2),pilot)].varValue*
             lines.loc[(go2,line2),'Requirement'] for go1,line1 in lines.index for go2,line2 in lines.index if go1 != go2 )
         for pilot in pilots.index
        )

63.44206349206348

In [30]:
display_side_by_side(output_df.loc[1:2,:],
                    output_df.loc[3:4,:],
                    output_df.loc[5:6,:],
                    output_df.loc[7:8,:],
                    output_df.loc[9,:])

PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 1 
 101 
 JACKAL 
 
 
 102 
 COPEN 
 
 
 103 
 DUKE 
 
 
 104 
 IRISH 
 
 
 105 
 REBEL 
 
 
 106 
 HEIST 
 
 
 107 
 SHADOW 
 
 
 108 
 WORM 
 
 
 2 
 201 
 BOND 
 
 
 202 
 SHINER 
 
 
 203 
 LITTLE 
 
 
 204 
 PITBULL 
 
 
 205 
 BUSTER 
 
 
 206 
 COACH 
 
 
 
 
 
 
 
 PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 3 
 301 
 HOLSTER 
 
 
 302 
 CLIP 
 
 
 303 
 HATTRICK 
 
 
 304 
 REBEL 
 
 
 305 
 TAOS 
 
 
 306 
 NAPE 
 
 
 307 
 SHADOW 
 
 
 308 
 COPEN 
 
 
 4 
 401 
 LITTLE 
 
 
 402 
 HEAT 
 
 
 403 
 DUKE 
 
 
 404 
 BOND 
 
 
 405 
 PITBULL 
 
 
 406 
 BUSTER 
 
 
 
 
 
 
 
 PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 5 
 501 
 FLIP 
 
 
 502 
 KONG 
 
 
 503 
 WORM 
 
 
 504 
 CLIP 
 
 
 505 
 HOOPS 
 
 
 506 
 COACH 
 
 
 507 
 NAPE 
 
 
 508 
 IRISH 
 
 
 6 
 601 
 COPEN 
 
 
 602 
 TAOS 
 
 
 603 
 HATTRICK 
 
 
 604 
 FACE 
 
 
 605 
 JACKAL 
 
 
 606 
 SKIDS 
 
 
 
 
 
 
 
 PILOT 
 
 
 Go 
 Line 
 
 
 
 
 
 7 
 701 
 DUKE 
 
 
 702 
 IRISH 
 
 
 703 
 PITBULL 
 
 
 704 
 CLIP 
 
 
 705 
 LITTLE 
 
 
 706 
 SHADOW 
 
 
 707 
 NAPE 
 
 
 708 
 AMP 
 
 
 8 
 801 
 TAOS 
 
 
 802 
 SHINER 
 
 
 803 
 COPEN 
 
 
 804 
 SKIDS 
 
 
 805 
 BUSTER 
 
 
 806 
 CASS 
 
 
 
 
 
 
 PILOT 
 
 
 Line 
 
 
 
 
 
 901 
 CLIP 
 
 
 902 
 REBEL 
 
 
 903 
 WORM 
 
 
 904 
 PITBULL 
 
 
 905 
 JACKAL 
 
 
 906 
 BOND 
 
 
 907 
 HOLSTER 
 
 
 908 
 NAPE

In [31]:
model

PilotMinSchedProb:
MAXIMIZE
-112*pilot_status_((1,_101),_'AMP') + -112*pilot_status_((1,_101),_'APE') + -112*pilot_status_((1,_101),_'BOND') + -112*pilot_status_((1,_101),_'BUSTER') + -112*pilot_status_((1,_101),_'CASS') + -112*pilot_status_((1,_101),_'CLIP') + -112*pilot_status_((1,_101),_'COACH') + -112*pilot_status_((1,_101),_'COPEN') + -112*pilot_status_((1,_101),_'CRUTCH') + -112*pilot_status_((1,_101),_'DUKE') + -112*pilot_status_((1,_101),_'FACE') + -112*pilot_status_((1,_101),_'FLIP') + -112*pilot_status_((1,_101),_'HATTRICK') + -112*pilot_status_((1,_101),_'HEAT') + -112*pilot_status_((1,_101),_'HEIST') + -112*pilot_status_((1,_101),_'HOLSTER') + -112*pilot_status_((1,_101),_'HOOPS') + -112*pilot_status_((1,_101),_'IRISH') + -112*pilot_status_((1,_101),_'JACKAL') + -112*pilot_status_((1,_101),_'KONG') + -112*pilot_status_((1,_101),_'LITTLE') + -112*pilot_status_((1,_101),_'NAPE') + -112*pilot_status_((1,_101),_'PHANTOM') + -112*pilot_status_((1,_101),_'PITBULL') + -112*pilot_s